### Text summarizer with Self trained model / function

<!-- ![hugging face](https://www.thesoftwarereport.com/wp-content/uploads/2023/09/Hugging-Face2.png) -->
Major Steps over covering the model 
- Scrape blog posts (medium, adobe) for net using beautiful soup 
- chunk them into sentences and summarize over these chunks (using the model to output over in text)

##### 1. Importing Dependencies 

In [132]:
import nltk
import os
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from bs4 import BeautifulSoup # scrap up blog post from the web 
import requests # make over http calls over to web (to scrap up a page)
import json

In [133]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Harman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Harman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Harman\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


##### 2. making web-scrapper

In [134]:
# URL = "https://hackernoon.com/simplifying-the-crazy-world-of-linux-distros"
URL = "https://medium.com/@palashm0002/understanding-the-basics-of-natural-language-processing-nlp-52b67cf954cb"
# URL = "https://hackernoon.com/nvidia-throws-gamers-under-the-bus"
# URL = "https://hackernoon.com/the-eggs-destined-to-give-birth-to-queens"

In [135]:
r = requests.get(URL)

In [136]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)
script_tag = soup.find('script', type='application/ld+json')

if script_tag:
    # Parse JSON data
    json_data = json.loads(script_tag.string)
    
    # Extract articleBody
    article_body = json_data.get('articleBody', '')

    print(article_body)

    ARTICLE = ARTICLE +". " + article_body
else:
    print("JSON-LD script tag not found.")

In [137]:
print(ARTICLE)

Sign up Sign in Sign up Sign in Understanding the Basics of Natural Language Processing (NLP) Palash Mishra Follow -- Listen Share Welcome to the fascinating world of Natural Language Processing (NLP)! NLP sits at the intersection of computer science, artificial intelligence, and linguistics, and it’s all about how computers can understand, interpret, and manipulate human language. This field is rapidly evolving, and its applications are vast, from search engines and voice assistants to sentiment analysis and machine translation. What is NLP? NLP involves a series of techniques designed to allow computers to process, analyze, and generate human language in a meaningful way. It’s not just about programming computers to understand the meaning of words; it’s about understanding context, tone, intent, and even the subtleties of humor and sarcasm. Key Concepts in NLP Tokenization This is the process of breaking text into individual terms or phrases. It can be as simple as splitting text bas

#### 3. Text propressing - and implementing TFIDF 

In [138]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words

In [139]:
def stem_words(words):
    stemmed_words = []
    for word in words:
       stemmed_words.append(stemmer.stem(word))
    return stemmed_words


In [140]:
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

In [141]:
tokenized_sentence = sent_tokenize(ARTICLE)
text = remove_special_characters(str(ARTICLE))
text = re.sub(r'\d+', '', text)

In [142]:
text

'Sign up Sign in Sign up Sign in Understanding the Basics of Natural Language Processing NLP Palash Mishra Follow  Listen Share Welcome to the fascinating world of Natural Language Processing NLP NLP sits at the intersection of computer science artificial intelligence and linguistics and its all about how computers can understand interpret and manipulate human language This field is rapidly evolving and its applications are vast from search engines and voice assistants to sentiment analysis and machine translation What is NLP NLP involves a series of techniques designed to allow computers to process analyze and generate human language in a meaningful way Its not just about programming computers to understand the meaning of words its about understanding context tone intent and even the subtleties of humor and sarcasm Key Concepts in NLP Tokenization This is the process of breaking text into individual terms or phrases It can be as simple as splitting text based on spaces and punctuation

In [143]:
tokenized_sentence

['Sign up Sign in Sign up Sign in Understanding the Basics of Natural Language Processing (NLP) Palash Mishra Follow -- Listen Share Welcome to the fascinating world of Natural Language Processing (NLP)!',
 'NLP sits at the intersection of computer science, artificial intelligence, and linguistics, and it’s all about how computers can understand, interpret, and manipulate human language.',
 'This field is rapidly evolving, and its applications are vast, from search engines and voice assistants to sentiment analysis and machine translation.',
 'What is NLP?',
 'NLP involves a series of techniques designed to allow computers to process, analyze, and generate human language in a meaningful way.',
 'It’s not just about programming computers to understand the meaning of words; it’s about understanding context, tone, intent, and even the subtleties of humor and sarcasm.',
 'Key Concepts in NLP Tokenization This is the process of breaking text into individual terms or phrases.',
 'It can be a

In [144]:
tokenized_words_with_stopwords = word_tokenize(text)
tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
tokenized_words = [word for word in tokenized_words if len(word) > 1]
tokenized_words = [word.lower() for word in tokenized_words]
tokenized_words = lemmatize_words(tokenized_words)

In [145]:
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq


In [146]:
print(tokenized_words)

['sign', 'sign', 'sign', 'sign', 'understanding', 'basic', 'natural', 'language', 'processing', 'nlp', 'palash', 'mishra', 'follow', 'listen', 'share', 'welcome', 'fascinating', 'world', 'natural', 'language', 'processing', 'nlp', 'nlp', 'sits', 'intersection', 'computer', 'science', 'artificial', 'intelligence', 'linguistics', 'computer', 'understand', 'interpret', 'manipulate', 'human', 'language', 'this', 'field', 'rapidly', 'evolving', 'application', 'vast', 'search', 'engine', 'voice', 'assistant', 'sentiment', 'analysis', 'machine', 'translation', 'what', 'nlp', 'nlp', 'involves', 'series', 'technique', 'designed', 'allow', 'computer', 'process', 'analyze', 'generate', 'human', 'language', 'meaningful', 'way', 'it', 'programming', 'computer', 'understand', 'meaning', 'word', 'understanding', 'context', 'tone', 'intent', 'even', 'subtlety', 'humor', 'sarcasm', 'key', 'concept', 'nlp', 'tokenization', 'this', 'process', 'breaking', 'text', 'individual', 'term', 'phrase', 'it', 'sim

In [147]:
word_freq = freq(tokenized_words)
input_user = int(input('Percentage of information to retain(in percent):'))
no_of_sentences = int((input_user * len(tokenized_sentence))/100)

In [148]:
print(word_freq)

{'sign': 4, 'understanding': 5, 'basic': 2, 'natural': 2, 'language': 8, 'processing': 2, 'nlp': 18, 'palash': 1, 'mishra': 1, 'follow': 1, 'listen': 1, 'share': 1, 'welcome': 1, 'fascinating': 1, 'world': 1, 'sits': 1, 'intersection': 1, 'computer': 4, 'science': 1, 'artificial': 2, 'intelligence': 2, 'linguistics': 1, 'understand': 2, 'interpret': 1, 'manipulate': 1, 'human': 2, 'this': 5, 'field': 6, 'rapidly': 1, 'evolving': 2, 'application': 2, 'vast': 1, 'search': 1, 'engine': 1, 'voice': 1, 'assistant': 1, 'sentiment': 2, 'analysis': 2, 'machine': 6, 'translation': 4, 'what': 1, 'involves': 2, 'series': 1, 'technique': 2, 'designed': 1, 'allow': 1, 'process': 2, 'analyze': 1, 'generate': 1, 'meaningful': 1, 'way': 1, 'it': 3, 'programming': 1, 'meaning': 3, 'word': 6, 'context': 1, 'tone': 1, 'intent': 1, 'even': 1, 'subtlety': 1, 'humor': 1, 'sarcasm': 2, 'key': 1, 'concept': 1, 'tokenization': 1, 'breaking': 1, 'text': 7, 'individual': 1, 'term': 2, 'phrase': 1, 'simple': 1, '

In [149]:
no_of_sentences

12

In [150]:
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb

In [151]:
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf

In [152]:
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf

def tf_idf_score(tf,idf):
    return tf*idf

In [153]:
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf

In [154]:
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     sentence = remove_special_characters(str(sentence)) 
     sentence = re.sub(r'\d+', '', sentence)
     pos_tagged_sentence = [] 
     no_of_sentences = len(sentences)
     pos_tagged_sentence = pos_tagging(sentence)
     for word in pos_tagged_sentence:
          if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                word = word.lower()
                word = wordlemmatizer.lemmatize(word)
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score

##### 6. Summarize Text

In [155]:
c = 1
sentence_with_importance = {}
for sent in tokenized_sentence:
    sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)    
    sentence_with_importance[c] = sentenceimp    
    c = c+1
sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)

In [156]:
sentence_with_importance

[(19, 0.11166524638537312),
 (5, 0.09113025585488264),
 (12, 0.07941512122148722),
 (26, 0.07588648835439675),
 (28, 0.06523593774598828),
 (27, 0.06486006099331693),
 (2, 0.06224394715254525),
 (13, 0.06162038487049087),
 (3, 0.061609167036286314),
 (6, 0.06121230967910766),
 (8, 0.05808459907910871),
 (17, 0.05470819461924675),
 (9, 0.05192799755732286),
 (14, 0.05044329914453654),
 (24, 0.05013007350737276),
 (11, 0.05009784977501907),
 (16, 0.048887984684778764),
 (29, 0.048524495054344724),
 (20, 0.045107744510323684),
 (10, 0.04489493226625007),
 (23, 0.03896541291033025),
 (18, 0.032411434716986744),
 (7, 0.03220819341647793),
 (22, 0.027870212353201175),
 (30, 0.018093711677779713),
 (25, 0.015369354617964757),
 (1, 0.007614027086183826),
 (4, 0.0),
 (15, 0.0),
 (21, 0.0)]

In [157]:
cnt = 0
summary = []
sentence_no = []
for word_prob in sentence_with_importance:
    if cnt < no_of_sentences:
        sentence_no.append(word_prob[0])
        cnt = cnt+1
    else:
      break
sentence_no.sort()
cnt = 1
for sentence in tokenized_sentence:
    if cnt in sentence_no:
       summary.append(sentence)
    cnt = cnt+1

summary = " ".join(summary)
# summary = summary.replace('&apos;s', '')
summary = summary.replace('&apos;', "'")
summary = summary.replace('&quot;', '"')


summary

'NLP sits at the intersection of computer science, artificial intelligence, and linguistics, and it’s all about how computers can understand, interpret, and manipulate human language. This field is rapidly evolving, and its applications are vast, from search engines and voice assistants to sentiment analysis and machine translation. NLP involves a series of techniques designed to allow computers to process, analyze, and generate human language in a meaningful way. It’s not just about programming computers to understand the meaning of words; it’s about understanding context, tone, intent, and even the subtleties of humor and sarcasm. It can be as simple as splitting text based on spaces and punctuation, or as complex as understanding compound words and idioms. Sentiment Analysis This is about determining the attitude or emotion of the speaker or writer. It’s widely used in analyzing opinions in social media, customer reviews, and survey responses. Statistical Methods The introduction of

In [158]:
len(ARTICLE), len(summary)

(3707, 1570)

In [159]:
with open('summary.txt', 'w') as f:
    f.write(summary)